In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 19/19 [00:03<00:00,  5.05it/s]


In [2]:
%matplotlib qt
import skimage.io as io
import pyFAI
from skimage.restoration import rolling_ball
import dask

In [3]:
scanid = 149248
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
#filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
dtype = np.int32

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15)
#test.map.estimate_background(method='rolling_ball', radius=10)
# Bad approximation
dark_field = XRDMap.from_image_stack(f'scan149567_dexela_xrd.tif', wd=filedir).map.max_image
test.map.subtract_background(background = dark_field)
flat_field = rolling_ball(median_filter(test.map.med_image, size=20), radius=10)

new_image_lst = []
for image in test.map.images.reshape(test.map.num_images, *test.map.image_shape):
    new_image = image / flat_field / np.median(image)
    new_image_lst.append(new_image)

test.map.images = np.asarray(new_image_lst).reshape(*test.map.shape)
test.map.dtype = np.float32

#test.map.subtract_background(background = background)
test.set_calibration('scan149562_dexela_calibration.poni')
#test.set_calibration('scan149256_dexela.poni')

In [4]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.images[1, 0])
fig.colorbar(im, ax=ax)
plt.show()

In [5]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.integrate_2d(test.map.images[1, 1], tth_num=1024, chi_num=1024)[0])
fig.colorbar(im, ax=ax)
plt.show()

In [223]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.images[0, 0])
fig.colorbar(im, ax=ax)
plt.show()

In [6]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
for filename, phase_name in zip(['AMCSD\\Hematite_0017806.cif',
                                 'AMCSD\\Iron-alpha_0011214.cif',
                                 'AMCSD\\Iron-beta_0011215.cif',
                                 'AMCSD\\Iron-delta_0011216.cif',
                                 'AMCSD\\Iron_0011146.cif',
                                 'AMCSD\\Cementite_0013523.cif',
                                 'AMCSD\\Zincite_0011555.cif'],
                                ['Hematite', 'Iron-ferrite', 'Iron-beta',
                                 'Iron-delta', 'Iron-austenite', 'Cementite',
                                 'Zincite']):
    
    test.load_phase(filename, filedir=cif_dir, phase_name=phase_name)

In [4]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="LiNbO3 161")
test.load_phase('Barker\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 148")
test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="Platinum")
test.load_phase('AMCSD\\Calcite_0000984.cif', filedir=cif_dir, phase_name="Calcite")
test.load_phase('AMCSD\\Aluminum_0011136.cif', filedir=cif_dir, phase_name="Aluminum")
test.load_phase('AMCSD\\Zincite_0011555.cif', filedir=cif_dir, phase_name="Zincite")
test.load_phase('AMCSD\\Gold_0011140.cif', filedir=cif_dir, phase_name="Gold")
test.load_phase('AMCSD\\Chromium_0011209.cif', filedir=cif_dir, phase_name="Chromium BCC")
test.load_phase('AMCSD\\Chromium_0011170.cif', filedir=cif_dir, phase_name="Chromium HCP")
test.load_phase('AMCSD\\Chromium_0011144.cif', filedir=cif_dir, phase_name="Chromium FCC")

XU.materials: Warning: element O used instead of O-2
XU.materials: Warning: element Nb used instead of Nb+5
XU.materials: Warning: element Li used instead of Li+1


In [355]:
'''test.phases = {}
test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="LiNbO3 161")
test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="Strained LiNbO3 161")
print(test.phases['Strained LiNbO3 161'].c)
test.phases['Strained LiNbO3 161'].ApplyStrain(np.zeros((3, 3)) + 0.001)
print(test.phases['Strained LiNbO3 161'].c)''';

XU.materials: Warning: element O used instead of O-2
XU.materials: Warning: element Nb used instead of Nb+5
XU.materials: Warning: element Li used instead of Li+1
XU.materials: Warning: element O used instead of O-2
XU.materials: Warning: element Nb used instead of Nb+5
XU.materials: Warning: element Li used instead of Li+1
13.4987
13.512212185208055


In [5]:
test.select_phases(remove_less_than=0.1)

In [6]:
test.calibrate_images()
background = rolling_ball(median_filter(test.map.med_image, size=20), radius=10)

  0%|          | 0/36 [00:00<?, ?it/s]

In [11]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.images[0, 0])
fig.colorbar(im, ax=ax)
plt.show()

In [19]:
#find_blobs(img, thresh, method='gaussian_threshold',
#               sigma=3,
#               tth=None, chi=None,
#               blob_expansion=5, min_blob_significance=500,
#               listme=False, plotme=False)

i = np.random.randint(test.map.images.shape[0])
j = np.random.randint(test.map.images.shape[1])
#i, j = 1, 2

#pixel = test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))[1]
pixel = test.map.images[i, j]

#thresh = 0.25 * np.std(background) # This should be taken from background, not the image
#thresh = np.median(test.map.images[i, j][test.map.images[i, j] != 0]) + 0.5 * np.std(background)
thresh = np.median(pixel[pixel != 0]) + 0.75 * np.std(background)

#blob_img, blob_mask = find_blobs(test.map.images[i, j], thresh = 3 * np.std(test.map.images[i, j]), sigma=1,
#                     tth=test.tth, chi=test.chi, plotme=True, min_blob_significance=100000)

#blob_df = find_blobs(test.map.images[i, j], thresh=thresh, sigma=3, blob_expansion=5,
#                     tth=test.tth, chi=test.chi, plotme=True, min_blob_significance=1000)
blob_df = find_blobs(pixel, thresh=thresh, sigma=3, blob_expansion=5, find_contours=True,
                     tth=test.tth, chi=test.chi, plotme=True,
                     min_blob_significance=1 * np.std(background) / test.tth_resolution / test.chi_resolution)
print(len(blob_df))

78


In [309]:
blob_df['significance']

0       [4.265503665291821]
1        [4.09792043735085]
2       [3.936619661145643]
3      [3.8320622008795038]
4      [3.7199030792141925]
               ...         
278     [3.100607068499699]
279    [3.0830899447476217]
280    [3.0404724004298185]
281    [3.0078206082405083]
282    [3.0035483336984776]
Name: significance, Length: 283, dtype: object

In [41]:
blob_df_lst = []
background = rolling_ball(median_filter(test.map.med_image, size=20), radius=10)

tic()
for pixel in tqdm(test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))):
    thresh = np.median(pixel[pixel != 0]) + 1 * np.std(background)
    blob_df = find_blobs(pixel, thresh=thresh, sigma=3, blob_expansion=5,
                     tth=test.tth, chi=test.chi, plotme=False, find_contours=False,
                     min_blob_significance=1 * np.std(background) / test.tth_resolution / test.chi_resolution)
    print(f'Found {len(blob_df)} blobs!')
    blob_df_lst.append(blob_df)
toc()

  0%|          | 0/36 [00:00<?, ?it/s]

Found 1 blobs!
Found 1 blobs!
Found 1 blobs!
Found 1 blobs!
Found 2 blobs!
Found 4 blobs!
Found 4 blobs!
Found 4 blobs!
Found 5 blobs!
Found 9 blobs!
Found 19 blobs!
Found 57 blobs!
Found 100 blobs!
Found 119 blobs!
Found 77 blobs!
Found 32 blobs!
Found 45 blobs!
Found 53 blobs!
Found 6 blobs!
Found 53 blobs!
Found 149 blobs!
Found 95 blobs!
Found 106 blobs!
Found 118 blobs!
Found 3 blobs!
Found 3 blobs!
Found 3 blobs!
Found 3 blobs!
Found 3 blobs!
Found 7 blobs!
Found 2 blobs!
Found 1 blobs!
Found 2 blobs!
Found 1 blobs!
Found 1 blobs!
Found 2 blobs!
dt = 47.235 sec


In [59]:
plot_value = []

for blob_df in blob_df_lst:
    #val = np.mean(blob_df['significance'])
    val = np.sum(blob_df['intensity'])
    #val = np.sum(np.hstack([*blob_df['value']]))
    plot_value.append(val)

plot_value = np.asarray(plot_value).reshape(*test.map.map_shape)

In [60]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_value, aspect='auto')
fig.colorbar(im, ax=ax)
plt.show()

In [27]:
from dask.diagnostics import ProgressBar
from tqdm.dask import TqdmCallback

In [32]:
def parallel_loop(function, iterable, *args, **kwargs):
    import dask

    # This may break with enumerate functions...
    
    @dask.delayed
    def sub_function(index):
        return function(index, *args, **kwargs)

    results = []
    for index in tqdm(iterable):
        results.append(sub_function(index, *args, **kwargs))

    #with ProgressBar():
    #    output = dask.compute(results)

    with TqdmCallback():
        output = dask.compute(results)

    return output[0]

In [33]:
def dask_find_blobs(pixel):
    thresh = np.median(pixel[pixel != 0]) + 1 * np.std(background)
    blob_df = find_blobs(pixel, thresh=thresh, sigma=3, blob_expansion=5,
                     tth=test.tth, chi=test.chi, plotme=False, find_contours=False,
                     min_blob_significance=1 * np.std(background) / test.tth_resolution / test.chi_resolution)
    return blob_df

In [34]:
iterable = test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))
blob_df_lst = parallel_loop(dask_find_blobs, iterable)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

C:\Users\emusterma\OneDrive - Brookhaven National Laboratory\Documents\Postdoc\Repositories\SRX_sXRD_analysis\testing_modules\sXRD_spot_blob_search.py:72: RuntimeWarning: invalid value encountered in divide
  blob_significance = blob_intensity / blob_area
c:\Users\emusterma\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\ndimage\_measurements.py:1535: RuntimeWarning: invalid value encountered in scalar divide
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


In [35]:
plot_value = []

for blob_df in blob_df_lst:
    #val = np.mean(blob_df['significance'])
    val = np.sum(blob_df['intensity'])
    #val = np.sum(np.hstack([*blob_df['value']]))
    plot_value.append(val)

plot_value = np.asarray(plot_value).reshape(*test.map.map_shape)

In [36]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_value, aspect='auto')
fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
blob_df_lst = []
background = rolling_ball(median_filter(test.map.med_image, size=20), radius=10)

tic()
for pixel in tqdm(test.map.images.reshape(test.map.num_images, len(test.chi), len(test.tth))):
    thresh = np.median(pixel[pixel != 0]) + 1 * np.std(background)
    blob_df = find_blobs(pixel, thresh=thresh, sigma=3, blob_expansion=5,
                     tth=test.tth, chi=test.chi, plotme=False, find_contours=False,
                     min_blob_significance=1 * np.std(background) / test.tth_resolution / test.chi_resolution)
    print(f'Found {len(blob_df)} blobs!')
    blob_df_lst.append(blob_df)
toc()

In [31]:
blob_df_lst[0][10]

,intensity,area,significance,value,mask,max_coords,center_of_mass
0,16.355833,[2.4402402368104283],[6.702550350111074],"[0.0036793833, 0.0031905272, 0.00320566, 0.003...","[[515, 656], [515, 657], [515, 658], [516, 653...","[25.439505097917078, 6.6736719151022115]","[25.444963294350952, 7.037203841848104]"
1,6.932606,[1.2640240222474104],[5.484552586207437],"[0.0026892584, 0.0029819047, 0.0031091243, 0.0...","[[1008, 443], [1008, 444], [1008, 445], [1008,...","[21.11675027871459, -15.990570958912343]","[21.115452979594316, -16.04766149709156]"
2,6.506614,[1.2354354336990037],[5.266656236257444],"[0.0036586255, 0.0033833215, 0.0032227158, 0.0...","[[326, 256], [326, 257], [326, 258], [327, 253...","[17.359589548005886, 18.283417708027258]","[17.339720517761506, 18.366781395399364]"
3,7.624613,[1.4825225204388044],[5.14299997467015],"[0.003209139, 0.0032824308, 0.0029235694, 0.00...","[[964, 24], [964, 25], [965, 21], [965, 22], [...","[12.592439373558284, -13.820053267191572]","[12.626638365180268, -14.086659589773927]"
4,3.846122,[0.8464264252367555],[4.543953437874125],"[0.0030859467, 0.0027998094, 0.0027127457, 0.0...","[[665, 442], [665, 443], [665, 444], [665, 445...","[21.11675027871459, 1.6764335085822992]","[21.1125841711614, 1.6467081249563051]"
5,4.723780,[1.1108708693095173],[4.2523215665183836],"[0.003023412, 0.0032367911, 0.0033327884, 0.00...","[[267, 443], [268, 440], [268, 441], [268, 442...","[21.076350700965037, 21.261569889690648]","[21.087393973517955, 21.309097147077907]"
6,4.162029,[1.0261261246838833],[4.056059668885686],"[0.0033229731, 0.0033094967, 0.003380662, 0.00...","[[845, 772], [845, 773], [845, 774], [845, 775...","[27.762480818516547, -8.469474771321767]","[27.778652001824522, -7.824441134576443]"
7,3.710619,[0.9321921908819756],[3.9805302463284624],"[0.003698356, 0.0035185765, 0.0034055123, 0.00...","[[795, 1306], [795, 1307], [795, 1308], [796, ...","[38.48856871102365, -5.390368278415558]","[38.529727238576, -5.14440172243329]"
8,2.410112,[0.6616216206916978],[3.6427352214726194],"[0.0032112433, 0.0029517028, 0.0034054932, 0.0...","[[421, 259], [422, 256], [422, 257], [422, 258...","[17.359589548005886, 13.790950857721484]","[17.35525241040647, 13.904013488424894]"
9,3.507345,[0.9771171157437575],[3.5894831658268878],"[0.0028345522, 0.0032207354, 0.0032644093, 0.0...","[[914, 1307], [914, 1308], [915, 1304], [915, ...","[38.50876849989844, -12.356215754170591]","[38.519040882691044, -11.5524310272306]"


In [405]:
elements_folder = f'output_tiff_scan2D_{scanid}_xs_sum8ch\\'

element_maps = {}
for element in os.listdir(f'{filedir}{elements_folder}'):
    el_im = io.imread(f'{filedir}{elements_folder}{element}')
    element_maps[str(element[:-5])] = el_im

In [406]:
key = 'detsum_Au_L'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(test.map.images, range(test.map.images.shape[0]), range(test.map.images.shape[1]), display_map=element_maps[key], display_title=key)

KeyError: 'detsum_Au_L'

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5), dpi=200)

im = ax.imshow(med_img)
fig.colorbar(im, ax=ax)

plt.show()

In [ ]:
ai = pyFAI.load(f'{filedir}scan149256_dexela.poni')
ai.energy = 15
tth_resolution = 0.02 # Degrees
chi_resolution = 0.05 # Degrees

res = ai.integrate2d_ng(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])[0], 100, 100, unit='2th_deg')
tth, chi = res[1], res[2]
tth_num = int(np.abs(np.max(tth) - np.min(tth)) // tth_resolution)
chi_num = int(np.abs(np.max(chi) - np.min(chi)) // chi_resolution)
# The bounds of interpolation should be limited by the intrument resolution AND the original image size. Should not make up too much data

# Is there a better way I could do this?
res = ai.integrate2d_ng(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])[0], tth_num, chi_num, unit='2th_deg')
tth, chi = res[1], res[2]
extent = [tth[0], tth[-1], chi[0], chi[-1]]

# Calibrate the full map. Can I parallelize this??
calibrated_map = np.zeros((proc_map.shape[0] * proc_map.shape[1], chi_num, tth_num), dtype=(dtype))
for i, pixel in enumerate(proc_map.reshape(proc_map.shape[0] * proc_map.shape[1], proc_map.shape[2], proc_map.shape[3])):
    res, _, _ = ai.integrate2d_ng(pixel, tth_num, chi_num, unit='2th_deg')
    calibrated_map[i] = res
calibrated_map = calibrated_map.reshape(proc_map.shape[0], proc_map.shape[1], chi_num, tth_num)

In [ ]:
key = 'detsum_Ni_K'
#interactive_dynamic_2d_plot(proc_map, range(121), range(121), img_vmin=10, img_vmax=100, display_map=element_maps['detsum_Pt_L'], map_vmin=1e4, map_vmax=1.1e4)
interactive_dynamic_2d_plot(calibrated_map, tth, chi, display_map=element_maps[key], display_title=key)